### Level 4 Variables

In [4]:
import pandas as pd
import numpy as np
import os

#### Economic Sensitivity (NCR Region Use Case)

In [6]:
def csvs_to_dfs(directory):
    dataframes = {}
    global_names = []

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            df_name = os.path.splitext(filename)[0].lower().replace(" ", "_")
            
            global_names.append(df_name)
            
            df = pd.read_csv(filepath)
            dataframes[df_name] = df
            print(f"Successfully read: {filename}")
    print("\n")
    
    if global_names:
        for name in global_names:
            print(name)
    
    return global_names, dataframes

df_names, dataframes = csvs_to_dfs(os.getcwd())

Successfully read: Active Establishments in the Locality.csv
Successfully read: Cost of Doing Business.csv
Successfully read: Cost of Living.csv
Successfully read: Employment Generation.csv
Successfully read: Financial Deepening.csv
Successfully read: Local Economy Growth.csv
Successfully read: Local Economy Size.csv
Successfully read: Presence of Business and Professional Organizations.csv
Successfully read: Productivity.csv
Successfully read: Safety Compliant Business.csv


active_establishments_in_the_locality
cost_of_doing_business
cost_of_living
employment_generation
financial_deepening
local_economy_growth
local_economy_size
presence_of_business_and_professional_organizations
productivity
safety_compliant_business


In [8]:
# SAMPLE
dataframes['active_establishments_in_the_locality'].head()

,PROVINCE / LGU,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Quezon (MM),-,-,-,0.9163,2.5000,2.2849,2.5000,2.5000,1.9815,1.7600
1,Caloocan,-,-,-,0.2244,0.3661,0.3552,0.5003,0.4275,0.4411,0.3098
2,Manila,-,-,-,1.6315,1.9449,2.2725,2.5000,1.8303,1.5532,0.0000
3,Pasay,-,-,-,0.1937,0.3809,0.3991,0.7650,0.5448,0.4364,0.0000
4,Paranaque,-,-,-,0.0000,0.6105,0.5298,0.9811,0.6202,0.5063,0.5142


In [10]:
def concat_year(year, df_names, dataframes):
    selected_dfs = []
    
    for name in df_names:
        df = dataframes[name]
        if year in df.columns:
            selected_df = df[["PROVINCE / LGU", year]].copy()
            selected_df.columns = ["PROVINCE / LGU", name]
            selected_dfs.append(selected_df.set_index("PROVINCE / LGU"))
    
    if selected_dfs:
        result_df = pd.concat(selected_dfs, axis=1)
        return result_df
    else:
        print("No valid data found for the specified year.")
        return None

df_2023 = concat_year('2023', df_names, dataframes)
# df_2023.to_csv("CMCI_2023_Full.csv", encoding="utf-8-sig")
df_2023.head()

,active_establishments_in_the_locality,cost_of_doing_business,cost_of_living,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,productivity,safety_compliant_business
PROVINCE / LGU,,,,,,,,,,
Quezon (MM),1.7600,1.4609,0.7925,0.9248,1.3115,0.5111,0.4807,0.1880,0.7522,1.2403
Caloocan,0.3098,1.3764,0.7925,0.1654,0.4734,0.3167,0.0555,0.0567,0.1673,0.8881
Manila,0.0000,1.2995,0.7925,0.4600,1.1539,0.3905,0.3591,0.3590,0.5652,0.6528
Pasay,0.0000,1.4822,0.7925,2.0000,0.5909,0.2022,2.0000,0.1095,2.0000,1.2830
Paranaque,0.5142,1.4267,0.7925,0.7071,0.3316,0.2952,0.3623,0.0081,0.5405,0.3482


In [12]:
df_2023.loc["Quezon (MM)"]

active_establishments_in_the_locality                  1.7600
cost_of_doing_business                                 1.4609
cost_of_living                                         0.7925
employment_generation                                  0.9248
financial_deepening                                    1.3115
local_economy_growth                                   0.5111
local_economy_size                                     0.4807
presence_of_business_and_professional_organizations    0.1880
productivity                                           0.7522
safety_compliant_business                              1.2403
Name: Quezon (MM), dtype: float64

In [14]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(df):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = df.columns
    vif_data["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return vif_data

df_vif_2023 = calculate_vif(df_2023)
df_vif_2023

,Feature,VIF
0,active_establishments_in_the_locality,11.382224
1,cost_of_doing_business,21.478783
2,cost_of_living,19.487497
3,employment_generation,9.472916
4,financial_deepening,9.180801
5,local_economy_growth,2.002097
6,local_economy_size,7.477743
7,presence_of_business_and_professional_organiza...,2.733491
8,productivity,13.954281
9,safety_compliant_business,10.562483


In [16]:
df_vif_2023[df_vif_2023["VIF"]<10]

,Feature,VIF
3,employment_generation,9.472916
4,financial_deepening,9.180801
5,local_economy_growth,2.002097
6,local_economy_size,7.477743
7,presence_of_business_and_professional_organiza...,2.733491


In [18]:
#Check for other years:
df_vif = []
for year in ["2017", "2018", "2019", "2020", "2021", "2022", "2023"]:
    vif_per_year = calculate_vif(concat_year(year, df_names, dataframes))
    vif_per_year.rename(columns={"VIF": year}, inplace=True)
    df_vif.append(vif_per_year.set_index("Feature"))
df_vif = pd.concat(df_vif, axis=1)
df_vif

,2017,2018,2019,2020,2021,2022,2023
Feature,,,,,,,
active_establishments_in_the_locality,5.375293,7.370544,7.764891,7.992509,12.487722,12.709310,11.382224
cost_of_doing_business,38.299912,61.435719,115.178282,7.162587,13.510331,60.874452,21.478783
cost_of_living,41.569627,60.044832,93.988044,9.881378,17.295669,62.602530,19.487497
employment_generation,4.413800,1.700540,7.921876,14.074284,11.482497,9.133051,9.472916
financial_deepening,5.006280,10.378706,7.589250,8.968114,13.358286,9.482077,9.180801
local_economy_growth,2.471867,5.855794,5.430889,6.385751,3.916462,2.701723,2.002097
local_economy_size,5.206583,2.899771,7.973866,14.839503,10.828888,7.430180,7.477743
presence_of_business_and_professional_organizations,3.011245,5.837291,2.364744,3.060768,2.574057,1.231398,2.733491
productivity,2.355058,2.534077,13.573734,37.326237,25.859109,18.911147,13.954281


In [20]:
def highlight_greater_than_10(s):
    is_greater = s > 10
    return ['background-color: pink' if v else '' for v in is_greater]
df_vif.style.apply(highlight_greater_than_10)

,2017,2018,2019,2020,2021,2022,2023
Feature,,,,,,,
active_establishments_in_the_locality,5.375293,7.370544,7.764891,7.992509,12.487722,12.709310,11.382224
cost_of_doing_business,38.299912,61.435719,115.178282,7.162587,13.510331,60.874452,21.478783
cost_of_living,41.569627,60.044832,93.988044,9.881378,17.295669,62.602530,19.487497
employment_generation,4.413800,1.700540,7.921876,14.074284,11.482497,9.133051,9.472916
financial_deepening,5.006280,10.378706,7.589250,8.968114,13.358286,9.482077,9.180801
local_economy_growth,2.471867,5.855794,5.430889,6.385751,3.916462,2.701723,2.002097
local_economy_size,5.206583,2.899771,7.973866,14.839503,10.828888,7.430180,7.477743
presence_of_business_and_professional_organizations,3.011245,5.837291,2.364744,3.060768,2.574057,1.231398,2.733491
productivity,2.355058,2.534077,13.573734,37.326237,25.859109,18.911147,13.954281


In [22]:
# At least two out of 2021 - 2023
cmci_shortlist = list(df_vif.index[np.r_[3:8, 9]])
cmci_2023 = df_2023[cmci_shortlist]
# cmci_2023.to_csv("CMCI_2023_Shortlist.csv", encoding="UTF-8-sig")
cmci_2023.head()

,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,safety_compliant_business
PROVINCE / LGU,,,,,,
Quezon (MM),0.9248,1.3115,0.5111,0.4807,0.1880,1.2403
Caloocan,0.1654,0.4734,0.3167,0.0555,0.0567,0.8881
Manila,0.4600,1.1539,0.3905,0.3591,0.3590,0.6528
Pasay,2.0000,0.5909,0.2022,2.0000,0.1095,1.2830
Paranaque,0.7071,0.3316,0.2952,0.3623,0.0081,0.3482


In [24]:
# Aggregate Using Arithmetic Mean
ncr_aggregated = cmci_2023.mean().to_frame().T
ncr_aggregated.index = ['NCR']

In [26]:
ncr_aggregated.head()

,employment_generation,financial_deepening,local_economy_growth,local_economy_size,presence_of_business_and_professional_organizations,safety_compliant_business
NCR,0.413952,0.50223,0.093069,0.225485,0.051037,0.602206
